# 1.5 '파인 튜닝'으로 정밀도 향상을 실현하는 방법 
- 이 파일은 이미 학습된 VGG 모델을 사용하여 파인 튜닝으로 개미와 벌의 이미지를 분류하는 모델을 학습합니다

# 학습 목표

1. PyTorch에서 GPU를 사용하는 구현 코드를 작성할 수 있습니다.
2. 최적화 기법의 설정에 있어서, 레이어마다 다른 학습률을 설정한 파인 튜닝을 구현할 수 있게 된다
3. 학습한 네트워크를 저장/로드할 수 있게 된다

# 사전 준비

- 1.4절에서 설명한 AWS EC2 GPU 인스턴스를 사용합니다.

In [3]:
# 패키지
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

In [4]:
# 시드 고정
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# DatasetとDataLoaderを作成

In [5]:
# 1.3 절에서 만든 클래스를 같은 폴더에있는 make_dataset_dataloader.py에 나열되어 사용
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

# 개미와 벌 이미지에 대한 파일 경로 목록 만들기
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

# Dataset 만들기
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')


# DataLoader작성
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 사전 객체로 결합
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


# 네트워크 모델 만들기

In [6]:
# 학습된 VGG-16 모델 로드

# VGG-16모델의 인스턴스 만들기
use_pretrained = True  # 학습된 매개변수 사용
net = models.vgg16(pretrained=use_pretrained)

# VGG16의 마지막 출력 레이어의 출력 유닛을 개미와 꿀 두 개로 교체
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)
print(net)

# 학습 모드로 설정
net.train()

print('네트워크 설정 완료: 학습된 가중치를 로드하고 훈련 모드로 설정했습니다.')


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# 손실 함수 정의

In [7]:
# 손실 함수 설정
criterion = nn.CrossEntropyLoss()

# 最適化手法を設定

In [8]:
# 파인 튜닝으로 학습시키는 파라미터를 변수 params_to_update의 1~3에 저장한다

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 학습할 계층의 매개 변수 이름 지정
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# 각 매개 변수에 대해 각 목록에 저장
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1에 저장：", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2에 저장：", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3에 저장：", name)

    else:
        param.requires_grad = False
        print("기울기 계산 없음. 학습하지 않음：", name)

params_to_update_1에 저장： features.0.weight
params_to_update_1에 저장： features.0.bias
params_to_update_1에 저장： features.2.weight
params_to_update_1에 저장： features.2.bias
params_to_update_1에 저장： features.5.weight
params_to_update_1에 저장： features.5.bias
params_to_update_1에 저장： features.7.weight
params_to_update_1에 저장： features.7.bias
params_to_update_1에 저장： features.10.weight
params_to_update_1에 저장： features.10.bias
params_to_update_1에 저장： features.12.weight
params_to_update_1에 저장： features.12.bias
params_to_update_1에 저장： features.14.weight
params_to_update_1에 저장： features.14.bias
params_to_update_1에 저장： features.17.weight
params_to_update_1에 저장： features.17.bias
params_to_update_1에 저장： features.19.weight
params_to_update_1에 저장： features.19.bias
params_to_update_1에 저장： features.21.weight
params_to_update_1에 저장： features.21.bias
params_to_update_1에 저장： features.24.weight
params_to_update_1에 저장： features.24.bias
params_to_update_1에 저장： features.26.weight
params_to_update_1에 저장： features.26.bias


In [9]:
# 최적화 기법 설정
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

# 학습 · 검증을 실시

In [10]:
# 모델을 학습시키는 함수 만들기
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # 초기 설정
    # GPU를 사용할 수 있는지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치：", device)

    # 네트워크를 GPU로
    net.to(device)

    # 네트워크가 어느 정도 고정되면 가속화
    torch.backends.cudnn.benchmark = True

    # epoch 루프
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epoch 당 훈련 및 검증 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 훈련 모드
            else:
                net.eval()   # 검증 모드

            epoch_loss = 0.0  # epoch 손실 합
            epoch_corrects = 0  # epoch의 정답

            # 학습되지 않은 검증 성능을 확인하기 위해 epoch = 0 훈련은 생략
            if (epoch == 0) and (phase == 'train'):
                continue

            # 데이터 로더에서 미니 배치를 꺼내는 루프
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # GPU를 사용할 수 있다면 GPU에 데이터를 보냅니다.
                inputs = inputs.to(device)
                labels = labels.to(device)

                # optimizer 초기화
                optimizer.zero_grad()

                # 순전파(forward) 계산
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 손실 계산
                    _, preds = torch.max(outputs, 1)  # 라벨 예측

                    # 오차역전파
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 결과 계산
                    epoch_loss += loss.item() * inputs.size(0)  # loss 합계 업데이트
                    # 정답의 합계 업데이트
                    epoch_corrects += torch.sum(preds == labels.data)

            # epoch 당 손실 및 정답률 표시
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

In [11]:
# 학습 및 검증 수행
num_epochs=2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

사용 장치： cuda:0
Epoch 1/2
-------------


100%|████████████████████████████████████████████| 5/5 [00:02<00:00,  1.93it/s]


val Loss: 0.7704 Acc: 0.4444
Epoch 2/2
-------------


100%|████████████████████████████████████████████| 8/8 [00:05<00:00,  1.60it/s]


train Loss: 0.5198 Acc: 0.7243


100%|████████████████████████████████████████████| 5/5 [00:01<00:00,  4.31it/s]

val Loss: 0.1657 Acc: 0.9542


# 학습한 네트워크 저장 및 로드

In [12]:
# PyTorch 네트워크 파라미터 저장
save_path = './weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)


In [13]:
# PyTorchのネットワークパラメータのロード
load_path = './weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# GPU上で保存された重みをCPU上でロードする場合
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)


<All keys matched successfully>

以上